In [7]:
import pandas as pd
import os

os.chdir('../')

In [8]:
def init_standard_death_cause(dir='data/字段解释.xlsx'):
    data = pd.read_excel(dir, sheet_name=2)
    # 将其转化为mapping dict (说明->{一级死因，二级，三级}
    death_cause_list = []
    for ii, d in enumerate(data['说明']):
        third = data.iloc[ii, 2]
        if not pd.isna(data.iloc[ii, 1]):
            second = data.iloc[ii, 1]
        if not pd.isna(data.iloc[ii, 0]):
            first = data.iloc[ii, 0]
        # if second
        death_cause_list.append((third, second, first, d))
    return death_cause_list
death_cause = init_standard_death_cause()

In [9]:
print(len(death_cause))
death_cause

72


[('宣告死亡', '宣告死亡', '宣告死亡', '指下落不明，法院以判决方式推定死亡'),
 ('自杀', '自杀', '自杀', '指蓄意结束自己生命'),
 ('他杀', '他杀', '意外死亡', '指被他人蓄意杀害'),
 ('跌倒坠落', '非交通意外', '意外死亡', '指因摔倒或高处坠落而死'),
 ('无生命机械力量', '非交通意外', '意外死亡', '指被物体砸死、压死、刺死、炸死等（不含被车撞死）'),
 ('有生命机械力量', '非交通意外', '意外死亡', '指被他人无意撞击、踩踏，被动物攻击等致死'),
 ('溺水', '非交通意外', '意外死亡', '指游泳、潜水、岸边游玩不慎落水而死（不含从船上落入水中）'),
 ('窒息', '非交通意外', '意外死亡', '指异物堵住呼吸道导致缺氧死亡'),
 ('火灾', '非交通意外', '意外死亡', '指被火烧死或在火灾事故中死亡'),
 ('烫伤', '非交通意外', '意外死亡', '指被高温蒸汽或液体重度烫伤后而亡'),
 ('电流、辐射和极度环境', '非交通意外', '意外死亡', '指触电、过量辐射、高温、低温致死'),
 ('自然灾害', '非交通意外', '意外死亡', '指地震、洪水、泥石流等自然灾害致死'),
 ('有毒动植物', '非交通意外', '意外死亡', '指食用有毒动植物，或被咬伤蜇伤致死'),
 ('有毒化学品', '非交通意外', '意外死亡', '指因煤气泄露、无意接触或食用有毒化学品等而死'),
 ('生命支持不足', '非交通意外', '意外死亡', '指体力过度消耗而死、饿死、渴死等'),
 ('医疗意外', '非交通意外', '意外死亡', '指医疗过程中发生意外而死'),
 ('行人', '交通意外', '意外死亡', '指步行时被其它交通工具碰撞致死'),
 ('非机动车', '交通意外', '意外死亡', '指骑或乘人力自行车、电动自行车、畜力车时发生事故而死'),
 ('二轮机动车', '交通意外', '意外死亡', '指骑或乘摩托车时发生事故而死'),
 ('三轮机动车', '交通意外', '意外死亡', '指骑或乘三轮机动车时发生事故而死'),
 ('轿车', '交通意外', '意外死亡', '指驾或乘不超过9座的乘用

In [10]:
claim_data = pd.read_excel('data/1.理赔数据.xlsx')
claim_cause = claim_data[['Reason1', 'Reason2']].apply(lambda x: '-'.join(x), axis=1).to_list()
claim_cause

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 7.0/7.0 MB 65.1 kB/s eta 0:00:00
     ------------------------------------- 224.5/224.5 kB 50.6 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 60.1 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


['因脑溢血身故-R99:其他原因不明确和未特指原因的死亡',
 '结肠癌 ;-C18.9:结肠恶性肿瘤',
 '因癌症身故 ;-C20:直肠恶性肿瘤',
 '因心肌梗身故-I46.1:心脏停搏',
 '肺结核 ;-R04.2:呼吸道出血',
 '癌性淋巴管炎-C49.9:其他结缔组织和软组织恶性肿瘤',
 '因病身故-C22.9:肝和肝内胆管恶性肿瘤',
 '(疾病身故 ;) 猝死-R99:其他原因不明确和未特指原因的死亡',
 '猝死，身故-R96.0:其他猝死，原因不知',
 '交通事故 ;身故-S06.9:颅内损伤',
 '心内猝死 ;-I46.1:心脏停搏',
 '（被保人因意外身故） 呼吸循环衰竭-R09.2:累及循环和呼吸系统的其他症状和体征',
 '(被保险人吐血于2014年10月15日进怀柔第一院就诊后转入北京地坛医院治疗) 肝癌-C22.9:肝和肝内胆管恶性肿瘤',
 '(脑出血后遗症身故)-I69.1:脑血管病后遗症',
 '死亡原因：中枢神经系统感染-G04.9:脑炎、脊髓炎和脑脊髓炎',
 '2014.02.20疾病身故(心源性猝死)-I46.1:心脏停搏',
 '急性胰腺炎导致身故； ;-K85:急性胰腺炎',
 '自缢身故； ;-R99:其他原因不明确和未特指原因的死亡',
 '脑干出血 ;-I61.3:脑内出血',
 '因急性心梗身故 ;-I21.9:急性心肌梗死',
 '11点55分死亡， ;（2015年12月25日因右肺上叶腺癌在医院死亡）-C34.9:支气管和肺恶性肿瘤',
 '因病身故-J18.8:肺炎，病原体未特指',
 '因颅内转移瘤病故-M8000/6:骨质疏松伴有病理性骨折',
 '因病身故-C22.9:肝和肝内胆管恶性肿瘤',
 '意外死亡-T75.1:其他外因的效应',
 '肝癌 ;-C22.9:肝和肝内胆管恶性肿瘤',
 '疾病身故-I21.9:急性心肌梗死',
 '跳楼身亡-R99:其他原因不明确和未特指原因的死亡',
 '在家中高坠身亡 ;-W17.9:从一个平面至另一平面的其他跌落造成的损伤',
 '上班途中突然身故-Y91.2:测定中毒水平以获得酒精影响的证据',
 '得大病后一直处于植物人状态，今早病发身故-R99:其他原因不明确和未特指原因的死亡',
 '交通事故导致身故 ;(颅脑损伤)-R99:其他

In [11]:
claim_data

,ID,ISSUE_date,OCCUR_DATE,GENDER,Issue_age,pay_dur,Province,City,Amount,Reason1,Reason2
0,120,2002-03-09,2015-04-04,M,31,20,11,1,6,因脑溢血身故,R99:其他原因不明确和未特指原因的死亡
1,230,2001-06-25,2016-01-01,F,49,20,11,1,1,结肠癌 ;,C18.9:结肠恶性肿瘤
2,496,2006-01-20,2017-06-12,M,16,30,11,1,4,因癌症身故 ;,C20:直肠恶性肿瘤
3,2212,2006-11-27,2018-06-25,M,43,20,11,1,5,因心肌梗身故,I46.1:心脏停搏
4,2446,2006-12-09,2018-11-03,M,43,30,11,1,3,肺结核 ;,R04.2:呼吸道出血
...,...,...,...,...,...,...,...,...,...,...,...
26199,5792935,2007-01-04,2015-06-11,M,47,20,52,4,2,因病身故,R99:其他原因不明确和未特指原因的死亡
26200,5793038,2007-09-14,2015-03-13,M,65,30,52,4,3,疾病身故。,R99:其他原因不明确和未特指原因的死亡
26201,5793667,2004-10-09,2015-12-14,M,40,20,52,2,1,癌症病故 ;,C80:部位未特指的恶性肿瘤
26202,5793790,2005-12-29,2018-04-30,M,53,15,52,2,2,疾病身故,Y89.9:其他外因的后遗症导致的疾病和死亡


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression

In [5]:
!pip install transformers

^C
